In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
lista_cidades = {}

In [3]:
estados = ['sp', 'mg']

In [4]:
dic = {
    " ": "_",
    "'": "",
    "â": "a",
    "ã": "a",
    "á": "a",
    "é": "e",
    "ê": "e",
    "í": "i",
    "ó": "o",
    "õ": "o",
    "ô": "o",
    "ú": "u",
    "ç": "c"
    
}

In [5]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [7]:
for estado in estados: 
    headers = {'user-agent': 'Mozilla/5.0'}
    url = f'https://www.feriados.com.br/feriados-estado-{estado}.php?ano=2023'
    response  = requests.get(url, headers = headers)
    response_text = response.text
    
    soup = BeautifulSoup(response_text, 'lxml')
    
    cidades = soup.find('div', id='div_outras_cidades').find_all('a')
    for c in cidades:
        cidade = c.text
        cidade_padronizado = replace_all(cidade.lower(), dic)
        lista_cidades[f'{cidade} - {estado.upper()}'] = f'{cidade_padronizado}-{estado}'

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
lista_cidades

In [ ]:
lista_completa = []

for nome, link in lista_cidades.items():
    
    lista_estado = []
    lista_cidade = []
    lista_datas = []
    lista_feriados = []

    
    url = f'https://www.feriados.com.br/feriados-{link}.php?ano=2023'

    
    response = requests.get(url)
    response_text = response.text

    soup = BeautifulSoup(response_text, 'lxml')

    feriads = soup.find('ul', class_='multi-column')
    feriados = feriads.find_all('li')


    for feriado in feriados:
        lista_cidade.append('-'.join(nome.split('-')[:-1]).strip())
        lista_estado.append(nome.split('-')[-1].strip())
        
        lista = feriado.text.split('-')
        lista_datas.append(lista[0].strip())
        lista_feriados.append((' '.join(lista[1:])).strip().title())

    lista_completa.extend(list(zip(lista_cidade,lista_estado,lista_datas,lista_feriados)))

In [ ]:
lista_completa

In [ ]:
columns = ['Cidade','Estado','Data','Feriado']

In [ ]:
df = pd.DataFrame(lista_completa, columns = columns)
df

pd.set_option('display.max_rows', None)

In [ ]:
df.head(10)

In [ ]:
df.Feriado.value_counts()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.Feriado.value_counts()

In [ ]:
df.drop(index = df[df['Feriado'] == 'Finados'].index, axis = 0, inplace=True)
df.drop(index = df[df['Feriado'] == 'Sexta Feira Da Paixão'].index, axis = 0, inplace=True)
df.drop(index = df[df['Feriado'] == 'Terça De Carnaval'].index, axis = 0, inplace=True)
df.replace('Dia Da Consciência Negra   Lei No. 3.249, De 18 De Novembro De 2008','Dia Da Consciência Negra', inplace=True)
df.replace('Dia Da Consciência Negra   Lei No 1.914 De 04 De Dezembro De 2007','Dia Da Consciência Negra', inplace=True)
df.replace('Dia Da Consciência Negra   Lei 4.731 22/12/2008.','Dia Da Consciência Negra', inplace=True)

In [ ]:
df.reset_index(drop=True)

In [ ]:
df[df['Feriado'] == 'Dia Da Consciência Negra   Lei 4.731 22/12/2008.']

In [ ]:
df.Feriado.value_counts()

In [ ]:
feriados_nacionais = ['Carnaval','Ano Novo','Nossa Senhora Aparecida','Natal','Proclamação Da República',
                      'Dia De Finados','Dia Do Servidor Público','Dia Do Professor','Independência Do Brasil',
                     'Corpus Christi','Dia Do Trabalho','Dia De Tiradentes','Sexta Feira Santa']

conditions = [
    (df['Feriado'].isin(feriados_nacionais)),
    (df['Feriado'] == 'Revolução Constitucionalista')
     ]

labels = ['Feriado Nacional', 'Feriado Estadual']

In [ ]:
df['Tipo'] = np.select(conditions, labels, default = 'Feriado Municipal')

In [ ]:
df

In [ ]:
df.to_csv('lista_feriados.csv', sep=';', encoding='UTF-8', index=False)